In [18]:
import json
import os
import glob

import os
import json
import numpy as np
import pandas as pd

In [17]:
def load_json_file(filepath):
    with open(filepath, 'r') as f:
        return json.load(f)

def collect_metrics(base_path, metric_name):
    utility_metrics = []
    for root, dirs, files in os.walk(base_path):
        for file in files:
            if file == f"{metric_name}_metrics.json":
                file_path = os.path.join(root, file)
                metrics = load_json_file(file_path)
                utility_metrics.append(metrics)
    return utility_metrics

def compute_means(dicts):
    result = dicts[0].copy()
    for k, v in dicts[0].items():
        if isinstance(v, dict):  # Check if the first element is a dict
            result[k] = compute_means([d[k] for d in dicts])
        else:
            result[k] = np.mean([d[k] for d in dicts])
    return result


In [31]:
models = [
    "TabDDPM", 
    "TabAutoDiff", 
    "StasyAutoDiff", 
    "CTGAN", 
    "Stasy",
    # "CTABGAN",
    ]

# Fidelity

In [35]:
fidelity_jsons = dict.fromkeys(models)
for model in models:
    base_dir = f'../results/{model}'
    fidelity_metrics_list = collect_metrics(base_dir, 'fidelity')
    mean_metrics = compute_means(fidelity_metrics_list)
    fidelity_jsons[model] = mean_metrics

In [78]:
fidelity_df = {}
for model, value in fidelity_jsons.items():
    syn = pd.DataFrame(value['syn_df']).reset_index().melt(id_vars='index', value_vars=['wasserstein_distance', 'jensenshannon_distance', 'euclidean_distance'], value_name='syn_df').dropna().set_index(['index', 'variable'])
    holdout = pd.DataFrame(value['holdout_df']).reset_index().melt(id_vars='index', value_vars=['wasserstein_distance', 'jensenshannon_distance', 'euclidean_distance'], value_name='holdout_df').dropna().set_index(['index', 'variable'])
    df = pd.concat([syn, holdout], axis=1)
    df['abs_diff'] = abs(df['syn_df'] - df['holdout_df'])
    # df['model'] = model
    fidelity_df[model] = df
fidelity_df = pd.concat(fidelity_df, axis=1)

fidelity_df


TabDDPM  \
                                                              syn_df   
index                              variable                            
visit_prob_per_store               wasserstein_distance     0.750667   
purchase_prob_per_product          wasserstein_distance     0.054761   
basket_size_per_product            wasserstein_distance    11.226417   
time_between_purchase_per_customer wasserstein_distance     9.718360   
customer_retention_per_store       wasserstein_distance     0.203385   
category_penetration_per_category  wasserstein_distance     0.389874   
retail_disc                        wasserstein_distance     6.290451   
coupon_disc                        wasserstein_distance     0.827523   
coupon_match_disc                  wasserstein_distance     0.096238   
revenue                            wasserstein_distance    15.038361   
quantity                           wasserstein_distance    12.172141   
age                                jensenshannon_distance   0.494244   
income                             jensenshannon_distance   0.338509   
household_size                     jensenshannon_distance   0.434221   
kids_count                         jensenshannon_distance   0.397373   
product_category                   jensenshannon_distance   0.319229   
department                         jensenshannon_distance   0.333908   
pearson_coef                       euclidean_distance       1.296608   
theils_u                           euclidean_distance       4.304366   
correl_ratio                       euclidean_distance       2.121117   

                                                                      \
                                                          holdout_df   
index                              variable                            
visit_prob_per_store               wasserstein_distance     0.003407   
purchase_prob_per_product          wasserstein_distance     0.000503   
basket_size_per_product            wasserstein_distance     0.348000   
time_between_purchase_per_customer wasserstein_distance     0.013092   
customer_retention_per_store       wasserstein_distance     0.000000   
category_penetration_per_category  wasserstein_distance     0.004876   
retail_disc                        wasserstein_distance     0.026817   
coupon_disc                        wasserstein_distance     0.003902   
coupon_match_disc                  wasserstein_distance     0.000335   
revenue                            wasserstein_distance     0.063883   
quantity                           wasserstein_distance     0.026500   
age                                jensenshannon_distance   0.534223   
income                             jensenshannon_distance   0.353115   
household_size                     jensenshannon_distance   0.481953   
kids_count                         jensenshannon_distance   0.394868   
product_category                   jensenshannon_distance   0.324565   
department                         jensenshannon_distance   0.345027   
pearson_coef                       euclidean_distance       0.459978   
theils_u                           euclidean_distance       0.036447   
correl_ratio                       euclidean_distance       0.046638   

                                                                      \
                                                            abs_diff   
index                              variable                            
visit_prob_per_store               wasserstein_distance     0.747260   
purchase_prob_per_product          wasserstein_distance     0.054259   
basket_size_per_product            wasserstein_distance    10.878417   
time_between_purchase_per_customer wasserstein_distance     9.705268   
customer_retention_per_store       wasserstein_distance     0.203385   
category_penetration_per_category  wasserstein_distance     0.384999   
retail_disc                        wasserstein_distance     6.263634   
coupon_disc                

In [79]:
fidelity_df.mean()

TabDDPM        syn_df          3.340388
               holdout_df      0.173406
               abs_diff        3.178858
TabAutoDiff    syn_df          1.600105
               holdout_df      0.141535
               abs_diff        1.461563
StasyAutoDiff  syn_df          4.843209
               holdout_df      0.159765
               abs_diff        4.689819
CTGAN          syn_df          1.741964
               holdout_df      0.164948
               abs_diff        1.584752
Stasy          syn_df        965.213814
               holdout_df      0.147753
               abs_diff      965.074029
dtype: float64

# Utility

In [32]:
utility_jsons = dict.fromkeys(models)
for model in models:
    base_dir = f'../results/{model}'
    utility_metrics_list = collect_metrics(base_dir, 'utility')
    mean_metrics = compute_means(utility_metrics_list)
    utility_jsons[model] = mean_metrics

In [33]:
# classification metrics
classification_df = []
for model, value in utility_jsons.items():
    df = pd.DataFrame(value['classification']).T
    df['model'] = model
    classification_df.append(df.reset_index())
pd.concat(classification_df)


,index,accuracy,f1,roc,precision,recall,model
0,train_df,0.653560,0.618877,0.676114,0.520915,0.762218,TabDDPM
1,holdout_df,0.657994,0.624367,0.681291,0.524954,0.770229,TabDDPM
2,syn_df,0.630144,0.088325,0.509422,0.488710,0.048550,TabDDPM
0,train_df,0.656398,0.622932,0.679793,0.523446,0.769108,TabAutoDiff
1,holdout_df,0.657580,0.624181,0.681030,0.524542,0.770550,TabAutoDiff
2,syn_df,0.674669,0.500091,0.626156,0.577545,0.440955,TabAutoDiff
0,train_df,0.654269,0.619757,0.676942,0.521563,0.763499,StasyAutoDiff
1,holdout_df,0.656102,0.623072,0.679792,0.523125,0.770229,StasyAutoDiff
2,syn_df,0.620092,0.128104,0.507077,0.418440,0.075629,StasyAutoDiff
0,train_df,0.651667,0.619420,0.675845,0.518943,0.768146,CTGAN


In [34]:
# regression metrics
classification_df = []
for model, value in utility_jsons.items():
    df = pd.DataFrame(value['regression']).T
    df['model'] = model
    classification_df.append(df.reset_index())
pd.concat(classification_df)


,index,mse,r2,model
0,train_df,2.309238e+02,-0.359233,TabDDPM
1,holdout_df,2.438209e+02,-0.435147,TabDDPM
2,syn_df,1.943164e+02,-0.143759,TabDDPM
0,train_df,2.361118e+02,-0.389770,TabAutoDiff
1,holdout_df,2.388025e+02,-0.405608,TabAutoDiff
2,syn_df,4.378087e+02,-1.576972,TabAutoDiff
0,train_df,2.383772e+02,-0.403104,StasyAutoDiff
1,holdout_df,2.399522e+02,-0.412375,StasyAutoDiff
2,syn_df,3.233087e+02,-0.903017,StasyAutoDiff
0,train_df,2.335960e+02,-0.374962,CTGAN


# Privacy

In [76]:
privacy_df = []
for model in models:
    base_dir = f'../results/{model}'
    privacy_metrics_list = collect_metrics(base_dir, 'privacy')
    mean_metrics = compute_means(privacy_metrics_list)
    mean_metrics['model'] = model
    privacy_df.append(mean_metrics)
privacy_df = pd.DataFrame(privacy_df)

privacy_df

,dcr,model
0,22.618683,TabDDPM
1,31.528440,TabAutoDiff
2,27.369472,StasyAutoDiff
3,31.417039,CTGAN
4,113.234550,Stasy
